In [1]:
import pandas as pd
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score as r2
from sklearn.linear_model import LinearRegression

In [2]:
musicnet_path = './input/musicnet'

data = pd.read_csv(musicnet_path + '/musicnet_metadata.csv')
data.head()

,id,composer,composition,movement,ensemble,source,transcriber,catalog_name,seconds
0,1727,Schubert,Piano Quintet in A major,2. Andante,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,447
1,1728,Schubert,Piano Quintet in A major,3. Scherzo: Presto,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,251
2,1729,Schubert,Piano Quintet in A major,4. Andantino - Allegretto,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,444
3,1730,Schubert,Piano Quintet in A major,5. Allegro giusto,Piano Quintet,European Archive,http://tirolmusic.blogspot.com/,OP114,368
4,1733,Schubert,Piano Sonata in A major,2. Andantino,Solo Piano,Museopen,Segundo G. Yogore,D959,546


In [3]:
path_train = musicnet_path + "/musicnet/train_labels"
path_test = musicnet_path + "/musicnet/test_labels"

split = []

for path in (path_train, path_test):
    all_files = glob.glob(path + "/*.csv")
    li = []
    for filename in all_files:
        df = pd.read_csv(filename)
        df['id'] = os.path.basename(filename[:-4])
        li.append(df)
    res = pd.concat(li, axis=0, ignore_index=True)
    res['id'] = res['id'].astype(str).astype(int)
    split.append(res)

In [4]:
train, test = split

In [5]:
drop_list = ['source', 'transcriber', 'catalog_name']
data_train = data.merge(train, on = 'id')
data_train = data_train.drop(drop_list, axis=1)

data_test = data.merge(test, on = 'id')
data_test = data_test.drop(drop_list, axis=1)

In [6]:
entire_data = pd.concat([data_test, data_train])
entire_data.head(5)

,id,composer,composition,movement,ensemble,seconds,start_time,end_time,instrument,note,start_beat,end_beat,note_value
0,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,90078,124382,1,63,0.0,1.000,Quarter
1,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,90078,124382,1,75,0.0,1.000,Quarter
2,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,90078,110558,1,48,0.0,0.375,Dotted Sixteenth
3,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,114654,122334,1,55,0.5,0.375,Dotted Sixteenth
4,1759,Schubert,Piano Sonata in C minor,3. Menuetto and Trio,Solo Piano,194,124382,139742,1,65,1.0,1.000,Quarter


In [7]:
data = entire_data.drop(['id', 'composition', 'movement', 'ensemble', 'instrument', 'note_value', 'composer'], axis=1)
data.head(5)

,seconds,start_time,end_time,note,start_beat,end_beat
0,194,90078,124382,63,0.0,1.000
1,194,90078,124382,75,0.0,1.000
2,194,90078,110558,48,0.0,0.375
3,194,114654,122334,55,0.5,0.375
4,194,124382,139742,65,1.0,1.000


In [8]:
X = data.drop(['note'], axis=1) # Features
y = data['note'] # Labels

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=10)

In [10]:
# Decision tree regressor
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

In [11]:
test = X_test.copy()
test['note'] = y_pred_tree
test = test.sort_values(by=['start_time'])
test.tail()

,seconds,start_time,end_time,start_beat,end_beat,note
922008,1067,46039006,46103518,2580.0,1.958333,77.333333
530130,1069,46069726,46291934,736.0,1.989583,74.750000
922015,1067,46141406,46219230,2583.0,1.958333,74.000000
922017,1067,46179806,46219230,2584.0,0.958333,43.000000
922023,1067,46286302,46617054,2586.0,4.958333,53.750000


In [12]:
r2_tree = r2(y_test, y_pred_tree)
print('Tree regression r^2:', r2_tree)

Tree regression r^2: 0.13966350993325138


In [68]:
song_id = 2303  # change this to another song id to get the predicted notes for different songs
data_sample = entire_data[entire_data['id'] == song_id]
data_sample = data_sample.drop(['id', 'composition', 'movement', 'ensemble', 'instrument', 'note_value', 'composer'], axis=1)

In [69]:
X_sample = data_sample.drop(['note'], axis=1)
y_sample = data_sample['note']
y_pred_tree_sample = tree.predict(X_sample)
print('Tree regression r^2 for row ', row, ':', r2(y_sample, y_pred_tree_sample))

Tree regression r^2 for row  2303 : 0.8077168603239916


In [70]:
import numpy as np
y_pred_tree_sample = y_pred_tree_sample.astype(int)
notes_pred_sample = pd.Series(y_pred_tree_sample)

In [71]:
from midiutil.MidiFile import MIDIFile

# if you get IndexError: pop from empty list, lower this maxlength. Its an issue with MIDIFile
maxlength = 90 # Max number of notes in track

midi_file = MIDIFile(numTracks=1)

#zip predicted midi note and start time and end time of note, and limit to maxlength
notes_sample = list(zip(notes_pred_sample, data_sample['start_time'], data_sample['end_time']))[:maxlength]

timescale = 0.4

filename = "output.mid"

# change timescale
for i in range(len(notes_sample)):
    tup = notes_sample[i]
    start_new = ((tup[1])/10000)*timescale
    end_new = ((tup[2])/10000)*timescale
    new_tup = (tup[0], start_new, end_new)
    notes_sample[i] = [round(x, 3) for x in new_tup]

    
velocity = 100
track = 0
channel = 0

# add MIDI notes to the file
for note, start_time, end_time in notes_sample:
    duration = end_time - start_time
    midi_file.addNote(track, channel, note, start_time, duration, velocity)

# write the MIDI file
with open(filename, "wb") as output_file:
    midi_file.writeFile(output_file)
